In [1]:
import pandas as pd
import requests
import os
import sqlite3
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
db_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'datalab.sqlite')
db_path

'/Users/paulcolas/code/RonanB400/decp_ml/data/datalab.sqlite'

In [3]:
conn = sqlite3.connect(db_path)
db = conn.cursor()
query = f"""
SELECT *
FROM "data.gouv.fr.2022.clean"
"""

data = pd.read_sql_query(query, conn)
data.columns

Index(['uid', 'id', 'nature', 'acheteur_id', 'acheteur_nom', 'acheteur_siren',
       'titulaire_id', 'titulaire_typeIdentifiant', 'titulaire_nom',
       'titulaire_siren', 'objet', 'montant', 'codeCPV', 'procedure',
       'dureeMois', 'dateNotification', 'datePublicationDonnees', 'formePrix',
       'attributionAvance', 'offresRecues', 'marcheInnovant', 'ccag',
       'sousTraitanceDeclaree', 'typeGroupementOperateurs', 'tauxAvance',
       'origineUE', 'origineFrance', 'lieuExecution_code',
       'lieuExecution_typeCode', 'idAccordCadre', 'source_open_data'],
      dtype='object')

In [4]:
acheteur_siren_df = data['acheteur_siren'].unique()
titulaire_siren_df = data['titulaire_siren'].unique()
print(len(acheteur_siren_df))
print(len(titulaire_siren_df))

10991
62841


In [5]:
data.head(2)

,uid,id,nature,acheteur_id,acheteur_nom,acheteur_siren,titulaire_id,titulaire_typeIdentifiant,titulaire_nom,titulaire_siren,...,ccag,sousTraitanceDeclaree,typeGroupementOperateurs,tauxAvance,origineUE,origineFrance,lieuExecution_code,lieuExecution_typeCode,idAccordCadre,source_open_data
0,2154005160001320242024-LOT04,20242024-LOT04,Marché,21540051600013,COMMUNE DE BATILLY,215400516,78885470100018,SIRET,ACOMETAL,788854701,...,Pas de CCAG,NaN,Pas de groupement,NaN,NaN,NaN,54980,Code postal,None,data.gouv.fr decp-2024.json
1,243500667002882021M226MO,2021M226MO,Marché,24350066700288,CC VAL D'ILLE-AUBIGNE,243500667,38373211200032,SIRET,UNIVERS,383732112,...,Prestations intellectuelles,1.0,Solidaire,NaN,NaN,NaN,35520,Code postal,None,data.gouv.fr decp-2024.json


In [6]:
# siren = '215400516'

# # Clé API fournie par INSEE
# api_key = "928d65d1-d6e4-49e1-8d65-d1d6e479e178"

# # URL de l'API Sirene
# url = f"https://api.insee.fr/api-sirene/3.11/siren/{siren}"

# # Headers avec la clé d'API
# headers = {
#     "X-INSEE-Api-Key-Integration": api_key
# }

# # Requête GET
# response = requests.get(url, headers=headers).json()

In [7]:
# effectif = response['uniteLegale']['trancheEffectifsUniteLegale']
# cat_entreprise = response['uniteLegale']['categorieEntreprise']
# response['header']['statut']

In [8]:
# api_key = "928d65d1-d6e4-49e1-8d65-d1d6e479e178"
# headers = {"X-INSEE-Api-Key-Integration": api_key}
# dico_acheteur_1 = {}

# for sir in acheteur_siren_df[:500]:
#     url = f"https://api.insee.fr/api-sirene/3.11/siren/{sir}"
#     try:
#         response = requests.get(url, headers=headers)
#         if response.status_code != 200:
#             continue

#         data = response.json()

#         unite_legale = data.get("uniteLegale")
#         if not unite_legale:
#             continue

#         tranche = unite_legale.get("trancheEffectifsUniteLegale")
#         if not tranche:
#                 continue
#         categorie = unite_legale.get("categorieEntreprise")
#         if not categorie:
#             continue

#         dico_acheteur_1[sir] = [tranche, categorie]

#     except Exception as e:
#         continue

In [9]:
# dico_acheteur_1

In [10]:
acheteur_siren_df[29]

'860200310'

In [11]:
# api_key = "928d65d1-d6e4-49e1-8d65-d1d6e479e178"
# headers = {"X-INSEE-Api-Key-Integration": api_key}
# dico_dico = {}

# for sir in acheteur_siren_df:
#     url = f"https://api.insee.fr/api-sirene/3.11/siren/{sir}"
#     response = requests.get(url, headers=headers).json()
#     effectif_categorie = [response['uniteLegale']['trancheEffectifsUniteLegale'], response['uniteLegale']['categorieEntreprise']]
#     dico_dico[sir] = effectif_categorie

In [12]:
dico_dico = {}

In [13]:
dict_effectif = {
    'NN' : "Unité non-employeuse ou présumée non-employeuse (faute de déclaration reçue)",
    '00' : "0 salarié (n'ayant pas d'effectif au 31/12 mais ayant employé des salariés au cours de l'année de référence)",
    '01' : "1 ou 2 salariés",
    '02' : "3 à 5 salariés",
    '03' : "6 à 9 salariés",
    '11' : "10 à 19 salariés",
    '12' : "20 à 49 salariés",
    '21' : "50 à 99 salariés",
    '22' : "100 à 199 salariés",
    '31' : "200 à 249 salariés",
    '32' : "250 à 499 salariés",
    '41' : "500 à 999 salariés",
    '42' : "1 000 à 1 999 salariés",
    '51' : "2 000 à 4 999 salariés",
    '52' : "5 000 à 9 999 salariés",
    '53' : "10 000 salariés et plus",
    'null' : 'donnée manquante ou "sans objet"'
}

In [14]:
# effectif = response['uniteLegale']['trancheEffectifsUniteLegale']
# cat_entreprise = response['uniteLegale']['categorieEntreprise']

In [15]:
# siren = '215400516'

# # Clé API fournie par INSEE
# api_key = "928d65d1-d6e4-49e1-8d65-d1d6e479e178"

# # URL de l'API Sirene
# url = 'https://api.insee.fr/api-sirene/3.11/siren/249710047'

# # Headers avec la clé d'API
# headers = {
#     "X-INSEE-Api-Key-Integration": api_key
# }

# # Requête GET
# response = requests.get(url, headers=headers).json()

In [16]:
# response

In [17]:
# 'https://api.insee.fr/api-sirene/3.11/siren?q={249710047,215400516}'

In [18]:
sirene_path = "/Users/paulcolas/code/sirene.csv"

In [19]:
'''
activitePrincipaleUniteLegale
anneeCategorieEntreprise
anneeEffectifsUniteLegale
caractereEmployeurUniteLegale
categorieEntreprise
categorieJuridiqueUniteLegale
dateCreationUniteLegale
dateDebut
dateDernierTraitementUniteLegale
denominationUniteLegale
denominationUsuelle1UniteLegale
denominationUsuelle2UniteLegale
denominationUsuelle3UniteLegale
economieSocialeSolidaireUniteLegale
etatAdministratifUniteLegale
identifiantAssociationUniteLegale
nicSiegeUniteLegale
nombrePeriodesUniteLegale
nomenclatureActivitePrincipaleUniteLegale
nomUniteLegale
nomUsageUniteLegale
prenom1UniteLegale
prenom2UniteLegale
prenom3UniteLegale
prenom4UniteLegale
prenomUsuelUniteLegale
pseudonymeUniteLegale
sexeUniteLegale
sigleUniteLegale
siren
societeMissionUniteLegale
statutDiffusionUniteLegale
trancheEffectifsUniteLegale
unitePurgeeUniteLegale
'''

'\nactivitePrincipaleUniteLegale\nanneeCategorieEntreprise\nanneeEffectifsUniteLegale\ncaractereEmployeurUniteLegale\ncategorieEntreprise\ncategorieJuridiqueUniteLegale\ndateCreationUniteLegale\ndateDebut\ndateDernierTraitementUniteLegale\ndenominationUniteLegale\ndenominationUsuelle1UniteLegale\ndenominationUsuelle2UniteLegale\ndenominationUsuelle3UniteLegale\neconomieSocialeSolidaireUniteLegale\netatAdministratifUniteLegale\nidentifiantAssociationUniteLegale\nnicSiegeUniteLegale\nnombrePeriodesUniteLegale\nnomenclatureActivitePrincipaleUniteLegale\nnomUniteLegale\nnomUsageUniteLegale\nprenom1UniteLegale\nprenom2UniteLegale\nprenom3UniteLegale\nprenom4UniteLegale\nprenomUsuelUniteLegale\npseudonymeUniteLegale\nsexeUniteLegale\nsigleUniteLegale\nsiren\nsocieteMissionUniteLegale\nstatutDiffusionUniteLegale\ntrancheEffectifsUniteLegale\nunitePurgeeUniteLegale\n'

In [20]:
colonnes_a_charger = ['categorieEntreprise', 'economieSocialeSolidaireUniteLegale', 'siren', 'trancheEffectifsUniteLegale']

In [21]:
sirene_df = pd.read_csv(sirene_path, usecols=colonnes_a_charger)

/var/folders/nj/s1c8f0tj3ksfjdg8xyxr6rdc0000gn/T/ipykernel_48447/1599831756.py:1: DtypeWarning: Columns (13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  sirene_df = pd.read_csv(sirene_path, usecols=colonnes_a_charger)


In [22]:
sirene_df.shape

(28340800, 4)

In [23]:
sirene_df.isna().sum()

siren                                         0
trancheEffectifsUniteLegale            26345830
categorieEntreprise                    18421557
economieSocialeSolidaireUniteLegale    22390803
dtype: int64

In [25]:
# sirene_df['check']
# sirene_df.head()

In [26]:
acheteur_siren_df = pd.DataFrame(acheteur_siren_df, columns=['siren'])
acheteur_siren_df.head()

,siren
0,215400516
1,243500667
2,249710047
3,625480199
4,200025633


In [28]:
acheteur_siren_df['siren'] = acheteur_siren_df['siren'].astype(str)
sirene_df['siren'] = sirene_df['siren'].astype(str)

# Refaire le merge
df_merge = acheteur_siren_df.merge(sirene_df, on='siren', how='left')

In [29]:
df_merge.shape

(10991, 4)

In [30]:
df_merge.head()

,siren,trancheEffectifsUniteLegale,categorieEntreprise,economieSocialeSolidaireUniteLegale
0,215400516,12,PME,NaN
1,243500667,22,PME,N
2,249710047,12,PME,N
3,625480199,22,ETI,N
4,200025633,12,PME,N


In [33]:
df_merge_acheteur = df_merge

In [32]:
titulaire_siren_df = pd.DataFrame(titulaire_siren_df, columns=['siren']).astype(str)

In [34]:
df_merge_titulaire = titulaire_siren_df.merge(sirene_df, on='siren', how='left')

In [35]:
df_merge_titulaire.head()

,siren,trancheEffectifsUniteLegale,categorieEntreprise,economieSocialeSolidaireUniteLegale
0,788854701,03,PME,NaN
1,383732112,03,PME,NaN
2,433872496,12,PME,N
3,552046955,53,GE,N
4,476980321,42,GE,N


In [36]:
df_merge_titulaire.isna().sum()

siren                                      0
trancheEffectifsUniteLegale             9142
categorieEntreprise                     3033
economieSocialeSolidaireUniteLegale    14852
dtype: int64

In [37]:
"""
amelioration : ajout d'une colonne check pour savoir si le merge a trouver ou non le numero de siren dans le big df.
permet de comprendre un NaN
"""

"\namelioration : ajout d'une colonne check pour savoir si le merge a trouver ou non le numero de siren dans le big df.\npermet de comprendre un NaN\n"